<a href="https://colab.research.google.com/github/ghoshatanu857/Instrument_Automation/blob/main/Vector_Magnetometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [192]:
import numpy as np
from plotly.subplots import make_subplots
from plotly import graph_objs as go
import plotly.figure_factory as ff

In [193]:
fig_template = go.layout.Template()
fig_template.layout = {
    'template': 'simple_white+presentation',
    'autosize': False,
    'width': 800,
    'height': 600,
    # 'opacity': 0.2,
    'xaxis': {
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white',
        },
    'yaxis': {
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white'
        },
    'font':{'family':'mathjax',
            'size': 22,
            }
}

In [262]:
def magField(current,distance,xy_points,mu):
  if type(xy_points)!=np.ndarray:
    field = (mu*current)/(2*np.pi*distance)
  else:
    field = np.ones((xy_points.shape[1],xy_points.shape[2]))
    field_above=[]; field_below=[]; coordinate_below =[]; coordinate_above =[]
    for i in range(xy_points.shape[1]):
      for j in range(xy_points.shape[2]):
        x=xy_points[:,i,j]
        if (x[0]==0 and x[1]==0):
          field[i,j] = None
        else:
          if x[1]<0:
            mu_below = mu*(1-(2.2e-5))
            field[i,j] = (mu_below*current)/(2*np.pi*np.sqrt(x[0]**2+x[1]**2))
            # print(f'for Negative Y : {x,field[i,j]}')
            field_below.append(field[i,j])
            coordinate_below.append(x)
          elif x[1]>0:
            field[i,j] = (mu*current)/(2*np.pi*np.sqrt(x[0]**2+x[1]**2))
            # print(f'for Positive Y : {x,field[i,j]}')
            field_above.append(field[i,j])
            coordinate_above.append(x)
    # print(f' Coordinate Below : {coordinate_below[-xy_points.shape[1]:]}')
    # print(f' Field Below : {field_below[-xy_points.shape[1]:]}')
    # print(f' Coordinate Above : {coordinate_above[0:xy_points.shape[1]]}')
    # print(f' Field Above : {field_above[0:xy_points.shape[1]]}')
    print(f'Field Comparison : {np.array(field_below[-xy_points.shape[1]:])<np.array(field_above[0:xy_points.shape[1]])}')
  return field

In [264]:
def circle(inner_r,outer_r,circle_points,line_points,mu):
  x_array=[]; y_array=[]
  if mu == None:
    print('It is for Coordinates.')
    x_array.append([points*np.cos(_points) for points,_points in zip(line_points,circle_points)])
    y_array.append([points*np.sin(_points) for points,_points in zip(line_points,circle_points)])
  else:
    mu_below = mu*(1-(2.2e-5))
    print('It is for Magnetic field.')
    for points, _points in zip(line_points,circle_points):
      x_array_points = []; y_array_points = []
      array_points = np.array(_points)
      below_pi = array_points[np.where(array_points<np.pi)]
      above_pi = array_points[np.where(array_points>np.pi)]
      print(f' Left Edge : {(points*np.sin(below_pi))[-1]>(points*(mu_below/mu)*np.sin(above_pi))[0]}\t \
       Right Edge : {(points*np.sin(below_pi))[0]>(points*(mu_below/mu)*np.sin(above_pi))[-1]}')

      x_array_points.append(np.concatenate((points*np.cos(below_pi),points*(mu_below/mu)*np.cos(above_pi))))
      y_array_points.append(np.concatenate((points*np.sin(below_pi),points*(mu_below/mu)*np.sin(above_pi))))
      x_array.append(x_array_points); y_array.append(y_array_points)
  return x_array, y_array

In [263]:
# def circle(inner_r,outer_r,circle_points,line_points,mu):
#   x_array=[]; y_array=[]
#   # if mu == None:
#   #   print('It is for Coordinates.')
#   #   x_array.append([points*np.cos(_points) for points,_points in zip(line_points,circle_points)])
#   #   y_array.append([points*np.sin(_points) for points,_points in zip(line_points,circle_points)])
#   # else:
#   # print('It is for Magnetic field.')
#   for points, _points in zip(line_points,circle_points):
#     x_array_points = []; y_array_points = []
#     array_points = np.array(_points)
#     below_pi = array_points[np.where(array_points<np.pi)]
#     above_pi = array_points[np.where(array_points>np.pi)]

#     if mu==None:
#       print('It is for Coordinates.')
#       x_array_points.append(np.concatenate((points*np.cos(below_pi),points*np.cos(above_pi))))
#       y_array_points.append(np.concatenate((points*np.sin(below_pi),points*np.sin(above_pi))))
#     else:
#       mu_below = mu*(1-(2.2e-5))
#       print('It is for Magnetic field.')
#       x_array_points.append(np.concatenate((points*np.cos(below_pi),points*(mu_below/mu)*np.cos(above_pi))))
#       y_array_points.append(np.concatenate((points*np.sin(below_pi),points*(mu_below/mu)*np.sin(above_pi))))
#       print(f' Left Edge : {(points*np.sin(below_pi))[-1]>(points*(mu_below/mu)*np.sin(above_pi))[0]}\t \
#       Right Edge : {(points*np.sin(below_pi))[0]>(points*(mu_below/mu)*np.sin(above_pi))[-1]}')
#     x_array.append(x_array_points); y_array.append(y_array_points)
#   return x_array, y_array

In [265]:
wire_radius = 25e-6; distance = 50e-6; wire_current = 10; mu_freeSpace = 4*np.pi*1e-7
linear_density = 30; init_angle = 0; final_angle = 2*np.pi
min_circular_density = 10; point_addition = 1

linear_points = np.linspace(wire_radius, distance, linear_density); inside_points = np.zeros((1))
circular_points=[]; circular_angles=[]
[circular_points.append(int(min_circular_density + circular_density*point_addition)) for circular_density in range(linear_density)]
circular_points = np.array(circular_points)
if np.all(circular_points)==False:
  raise Exception('Check the values of circular_points')
circular_angles = [np.linspace(init_angle, final_angle, _points, endpoint=False) for _points in circular_points]

mag_field = magField(current=wire_current, xy_points=False,distance=linear_points,mu=mu_freeSpace)
x_points, y_points = circle(wire_radius, distance, circular_angles, mu=None,line_points=np.concatenate((inside_points,linear_points)))

It is for Coordinates.


In [268]:
x_points[0][0].shape,y_points[0][0].shape,B_x[0][0].shape,B_y[0][0].shape

((10,), (10,), (9,), (9,))

In [267]:
total_distance = np.concatenate((inside_points,linear_points))
Bnew = magField(current=wire_current,xy_points=False, distance=total_distance,mu=mu_freeSpace)
Bnew[np.where(np.isinf(Bnew))[0]]=0
B_x,B_y = circle(wire_radius, distance,circular_angles,mu=mu_freeSpace,line_points=Bnew)

It is for Magnetic field.
 Left Edge : False	        Right Edge : False
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Left Edge : True	        Right Edge : True
 Lef

<ipython-input-262-043a748558dc>:3: RuntimeWarning:

divide by zero encountered in divide



In [278]:
x_linear_point = np.concatenate((-linear_points[::-1],inside_points,linear_points))
y_linear_point = x_linear_point+wire_radius

X, Y = np.meshgrid(x_linear_point, y_linear_point)

B = magField(current=wire_current, distance=None, mu= mu_freeSpace,xy_points=np.array([X,Y]))

Field Comparison : [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False]


In [280]:
y_linear_point

array([-2.50000000e-05, -2.41379310e-05, -2.32758621e-05, -2.24137931e-05,
       -2.15517241e-05, -2.06896552e-05, -1.98275862e-05, -1.89655172e-05,
       -1.81034483e-05, -1.72413793e-05, -1.63793103e-05, -1.55172414e-05,
       -1.46551724e-05, -1.37931034e-05, -1.29310345e-05, -1.20689655e-05,
       -1.12068966e-05, -1.03448276e-05, -9.48275862e-06, -8.62068966e-06,
       -7.75862069e-06, -6.89655172e-06, -6.03448276e-06, -5.17241379e-06,
       -4.31034483e-06, -3.44827586e-06, -2.58620690e-06, -1.72413793e-06,
       -8.62068966e-07,  0.00000000e+00,  2.50000000e-05,  5.00000000e-05,
        5.08620690e-05,  5.17241379e-05,  5.25862069e-05,  5.34482759e-05,
        5.43103448e-05,  5.51724138e-05,  5.60344828e-05,  5.68965517e-05,
        5.77586207e-05,  5.86206897e-05,  5.94827586e-05,  6.03448276e-05,
        6.12068966e-05,  6.20689655e-05,  6.29310345e-05,  6.37931034e-05,
        6.46551724e-05,  6.55172414e-05,  6.63793103e-05,  6.72413793e-05,
        6.81034483e-05,  

In [284]:
fig = go.Figure(data=go.Contour(x=x_linear_point,y=y_linear_point,z=B,contours_coloring='heatmap',
                                connectgaps = True,line_smoothing=1,colorbar=dict(title='Magnetic Field',titleside='right'),colorscale=None))
fig.update_traces(line_width=0)
# fig.add_trace(go.Scatter(x=wire_radius*np.cos(np.linspace(0,2*np.pi,100)), y=wire_radius*np.sin(np.linspace(0,2*np.pi,100)),
#                          mode='lines', line=dict(color='yellow', width=0.5),fill = 'tonexty',fillcolor = 'rgba(135, 206, 250, 0.3)'))

# for radi in range(0,len(x_points[0]),10):
#     x = x_points[0][radi]; y = y_points[0][radi]
#     # x = np.squeeze(np.array(x_points[radi])); y = np.squeeze(np.array(y_points[radi]))
#     u = -np.squeeze(np.array(B_y[radi])); v = np.squeeze(np.array(B_x[radi]))
#     fig1 = ff.create_quiver(x, y, u, v, scale=0.0001, arrow_scale=0.5, scaleratio=1.0, angle=np.pi/9, line=dict(width=1.2, color='black'))

#     # u = -y; v = x
#     # fig1 = ff.create_quiver(x, y, u, v, scale=125e-6*mag_field[radi]/linear_points[radi], arrow_scale=0.5, scaleratio=1.0, angle=np.pi/9, line=dict(width=1.2, color='black'))
#     fig.add_traces(data=fig1.data)

# fig.add_trace(go.Scatter(x=[0], y=[0], mode='markers', marker_size=15, marker=dict(color='yellow')))
# fig.add_trace(go.Scatter(x=linear_point, y=-wire_radius*np.ones(linear_point.shape[0]), mode='lines', marker=dict(color='yellow'),fill=None))
# fig.add_trace(go.Scatter(x=linear_point, y=-distance*np.ones(linear_point.shape[0]), mode='lines', marker=dict(color='grey'),
#                          fill='tonexty',fillcolor = 'rgba(255, 255, 255, 0.4)'))

fig.update_layout(showlegend=False)
fig.update_layout(template=fig_template, title = None,width=800, height=700)
# fig.update_xaxes(title = 'Distance',range = [-distance,distance]); fig.update_yaxes(title = 'Distance', range = [-distance,distance])
fig.write_html('MagField_RadDistance.html')
fig.show()